# Text generation with LSTM

In [1]:
import keras

keras.__version__

2022-10-23 12:28:21.744530: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 12:28:25.052229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-23 12:28:25.052278: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-23 12:28:25.396317: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-23 12:28:32.088966: W tensorflow/stream_executor/pla

'2.10.0'

## Preparing the data

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)
text = open(path).read().lower()
print("Corpus length:", len(text))

Corpus length: 600893


In [3]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print("Unique characters:", len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print("Vectorization...")
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


/tmp/ipykernel_14664/2567106722.py:26: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
/tmp/ipykernel_14664/2567106722.py:27: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


## Building the network

In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation="softmax"))

2022-10-23 12:28:42.420954: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-23 12:28:42.421002: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-23 12:28:42.421036: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-f22901): /proc/driver/nvidia/version does not exist
2022-10-23 12:28:42.439265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

/home/codespace/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## Training the language model and sampling from it

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import random
import sys

for epoch in range(1, 60):
    print("epoch", epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y, batch_size=128, epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index : start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print("------ temperature:", temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.0

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1


2022-10-23 12:28:43.534587: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 684950760 exceeds 10% of free system memory.


1565/1565 [==============================] - 156s 97ms/step - loss: 1.9690
--- Generating with seed: "transition from religion by way of science entails a powerfu"
------ temperature: 0.2
transition from religion by way of science entails a powerful of the conscience of the consequently the consequently and general and the consequently and sense of the consequently and the consequently the sense of the conseral and the consertion of the consertion of the sense of the consertion of the consequently and the sense of the consequently and the former the conserder the conseral and the consertion of the consequently sense of the consequently some
------ temperature: 0.5
onsertion of the consequently sense of the consequently some the philosophice the sense and enthere of the for the conscience of the sense and conserre and we disting and very have for the fair for the intere it it has former, and the netefound, this is the can conserration and though of and the condition of and the that the 

2022-10-23 12:33:02.600670: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 684950760 exceeds 10% of free system memory.


1565/1565 [==============================] - 148s 94ms/step - loss: 1.6141
--- Generating with seed: "this sentiment be
left to men of the masses who stand in nee"
------ temperature: 0.2
this sentiment be
left to men of the masses who stand in needs all the self-course of the self-castions of the self-conscience of the self-consequently instincts, and the self-case of the self-consequently and stronger and delight and stronger and all the self-course of the self-conscience, and the sense of the same are all the stronger and more instinct of the self-coural and some to self-creater and all the self-conscience of the self-course of the self-
------ temperature: 0.5
 and all the self-conscience of the self-course of the self-case of the believes have been seems to all been does the self-concerning the fulless for the stronger of his instinct for the more every sentimentally of the self-nothing the will to seems of philosophy, which has no more but an etrary latter and self inasmisching i

2022-10-23 12:37:01.386661: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 684950760 exceeds 10% of free system memory.


 516/1565 [========>.....................] - ETA: 1:44 - loss: 1.5210

: 

: 